# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Oct 26 2022  3:28PM  249462        10  0086119092  ISDIN Corporation   
1  Oct 26 2022  3:28PM  249462        10  0086119093  ISDIN Corporation   
2  Oct 26 2022  3:28PM  249462        10  0086119107  ISDIN Corporation   
3  Oct 26 2022  3:28PM  249462        10  0086119108  ISDIN Corporation   
4  Oct 26 2022  3:28PM  249462        10  0086119115  ISDIN Corporation   
5  Oct 26 2022  3:22PM  249455        10  0086110227  ISDIN Corporation   
6  Oct 26 2022  3:22PM  249455        10  0086110232  ISDIN Corporation   
7  Oct 26 2022  3:22PM  249455        10  0086110256  ISDIN Corporation   
8  Oct 26 2022  3:22PM  249455        10  0086110274  ISDIN Corporation   
9  Oct 26 2022  3:22PM  249455        10  0086110275  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
12  249457       Released         16
13  249459       Released          1
14  249460       Released          1
15  249461       Released          1
16  249462       Released          5

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
249457                NaN        NaN      16.0
249459                NaN        NaN       1.0
249460                NaN        NaN       1.0
249461                NaN        NaN       1.0
249462                NaN        NaN       5.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249382                0.0        0.0       1.0
249407                0.0        0.0       1.0
249408                0.0        0.0       1.0
249418                0.0        0.0       1.0
249447                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
249382                  0          0         1
249407                  0          0         1
249408                  0          0         1
249418                  0          0         1
249447                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               249382          0          0         1
1               249407          0          0         1
2               249408          0          0         1
3               249418          0          0         1
4               249447          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               249382                             1
1               249407                             1
2               249408                             1
3               249418                             1
4               249447                             1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse             Customer
0   Oct 26 2022  3:28PM  249462        10    ISDIN Corporation
5   Oct 26 2022  3:22PM  249455        10    ISDIN Corporation
21  Oct 26 2022  3:08PM  249461        22    NBTY Global, Inc.
22  Oct 26 2022  3:07PM  249460        22    NBTY Global, Inc.
23  Oct 26 2022  3:04PM  249459        12     SD Head USA, LLC
24  Oct 26 2022  2:40PM  249457        10    ISDIN Corporation
40  Oct 26 2022  2:10PM  249454        10    ISDIN Corporation
43  Oct 26 2022  1:51PM  249453        10    ISDIN Corporation
51  Oct 26 2022  1:23PM  249451        21    NBTY Global, Inc.
52  Oct 26 2022  1:20PM  249450        19  AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse             Customer Completed  \
0  Oct 26 2022  3:28PM  249462        10    ISDIN Corporation             
1  Oct 26 2022  3:22PM  249455        10    ISDIN Corporation             
2  Oct 26 2022  3:08PM  249461        22    NBTY Global, Inc.             
3  Oct 26 2022  3:07PM  249460        22    NBTY Global, Inc.             
4  Oct 26 2022  3:04PM  249459        12     SD Head USA, LLC             
5  Oct 26 2022  2:40PM  249457        10    ISDIN Corporation             
6  Oct 26 2022  2:10PM  249454        10    ISDIN Corporation             
7  Oct 26 2022  1:51PM  249453        10    ISDIN Corporation             
8  Oct 26 2022  1:23PM  249451        21    NBTY Global, Inc.             
9  Oct 26 2022  1:20PM  249450        19  AdvaGen Pharma, Ltd         2   

  Executing  Released  
0                   5  
1                  16  
2                   1  
3                   1  
4                   1  
5                  16  
6                   3  
7                   8  
8                   1  
9        33         2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse             Customer  Released  \
0  Oct 26 2022  3:28PM  249462        10    ISDIN Corporation         5   
1  Oct 26 2022  3:22PM  249455        10    ISDIN Corporation        16   
2  Oct 26 2022  3:08PM  249461        22    NBTY Global, Inc.         1   
3  Oct 26 2022  3:07PM  249460        22    NBTY Global, Inc.         1   
4  Oct 26 2022  3:04PM  249459        12     SD Head USA, LLC         1   
5  Oct 26 2022  2:40PM  249457        10    ISDIN Corporation        16   
6  Oct 26 2022  2:10PM  249454        10    ISDIN Corporation         3   
7  Oct 26 2022  1:51PM  249453        10    ISDIN Corporation         8   
8  Oct 26 2022  1:23PM  249451        21    NBTY Global, Inc.         1   
9  Oct 26 2022  1:20PM  249450        19  AdvaGen Pharma, Ltd         2   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9        33         2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Oct 26 2022  3:28PM  249462        10  ISDIN Corporation         5   
1  Oct 26 2022  3:22PM  249455        10  ISDIN Corporation        16   
2  Oct 26 2022  3:08PM  249461        22  NBTY Global, Inc.         1   
3  Oct 26 2022  3:07PM  249460        22  NBTY Global, Inc.         1   
4  Oct 26 2022  3:04PM  249459        12   SD Head USA, LLC         1   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Oct 26 2022  3:28PM  249462        10  ISDIN Corporation         5   
1  Oct 26 2022  3:22PM  249455        10  ISDIN Corporation        16   
2  Oct 26 2022  3:08PM  249461        22  NBTY Global, Inc.         1   
3  Oct 26 2022  3:07PM  249460        22  NBTY Global, Inc.         1   
4  Oct 26 2022  3:04PM  249459        12   SD Head USA, LLC         1   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Oct 26 2022  3:28PM  249462        10  ISDIN Corporation         5   
1  Oct 26 2022  3:22PM  249455        10  ISDIN Corporation        16   
2  Oct 26 2022  3:08PM  249461        22  NBTY Global, Inc.         1   
3  Oct 26 2022  3:07PM  249460        22  NBTY Global, Inc.         1   
4  Oct 26 2022  3:04PM  249459        12   SD Head USA, LLC         1   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1247281        48        0.0        0.0
12          498877         2        0.0        0.0
19          748279         4       33.0        2.0
21          498858         2        0.0        0.0
22          748329         3        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1247281        48        0.0        0.0
1        12   498877         2        0.0        0.0
2        19   748279         4       33.0        2.0
3        21   498858         2        0.0        0.0
4        22   748329         3        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10        48        0.0        0.0
1        12         2        0.0        0.0
2        19         4       33.0        2.0
3        21         2        0.0        0.0
4        22         3        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   48.0
1        12  Released    2.0
2        19  Released    4.0
3        21  Released    2.0
4        22  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    19   21   22
Status                              
Completed   0.0  0.0   2.0  0.0  0.0
Executing   0.0  0.0  33.0  0.0  0.0
Released   48.0  2.0   4.0  2.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    19   21   22
0          Completed   0.0  0.0   2.0  0.0  0.0
1          Executing   0.0  0.0  33.0  0.0  0.0
2           Released  48.0  2.0   4.0  2.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    19   21   22
0  Completed   0.0  0.0   2.0  0.0  0.0
1  Executing   0.0  0.0  33.0  0.0  0.0
2   Released  48.0  2.0   4.0  2.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()